In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from beamforming import construct_slowness_grid, construct_times_beamforming, precompute_A, CMTM, noise_space_projection


In [3]:


Ntheta, Nslow = 30, 20  # Resolution of the slowness grid
vmin, vmax = 500, 10_000  # Velocity range of slowness grid
theta = np.linspace(0, 2 * np.pi, Ntheta)  # Azimuthal range of slowness grid

""" Generate random receiver locations (replace with DAS UTM coords) """
RNG = np.random.default_rng(42)
Nch = 100  # Number of receivers
x, y = RNG.uniform(-10, 10, size=(2, Nch))  # x/y are in the same units of distance as vmin/vmax
print(x.shape)

""" Generate random data (replace with DAS data) """
samp = 100.
Nt = 1024
data = RNG.normal(size=(Nch, Nt))

""" Select frequency band """
freq_band = (1.0, 2.0)  # 1-2 Hz. Must be a narrow range, but not too narrow...
NFFT = 2**int(np.log2(Nt) + 1) + 1  # Don't mess with this...
freqs = scipy.fft.rfftfreq(n=2*NFFT, d=1/samp)
inds = (freqs >= freq_band[0]) & (freqs < freq_band[1])
freqs_select = freqs[inds]

""" Initialise beamformer """
# Slowness in x/y, and velocity grid
Sx, Sy, v_grid = construct_slowness_grid(theta, vmin, vmax, Nslow)
Sx = Sx.ravel().reshape((1, -1))
Sy = Sy.ravel().reshape((1, -1))
print(Sx.shape)
# Differential times
dt = construct_times_beamforming(x, y, Sx, Sy)
# Steering vectors
A = precompute_A(dt, freqs_select)

(100,)
(1, 600)


In [4]:
""" Compute covariance matrix """
Nw = 3  # Number of tapers
Cxy = CMTM(data, Nw=Nw, freq_band=freq_band, fsamp=samp)

""" Compute beampower """
Pr = noise_space_projection(Cxy, A, sources=1)
P = 1.0 / Pr

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
P2 = P.reshape((Nslow, Ntheta))

